<a href="https://colab.research.google.com/github/shardul28/Document-similarity-K-shingles-minhashing-LSH-python/blob/master/FINALLLLL83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [85]:
!pip install mit-news-classify

In [86]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [87]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import pickle
from mitnewsclassify import tfidf as mitx
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.util import ngrams
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict,Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
cd /content/drive/MyDrive/project_data

/content/drive/MyDrive/project_data


# Fetching Data from Drive

In [90]:
infile = open('ndtv_data.pkl','rb')
ndtv_df = pickle.load(infile)

In [91]:
ndtv_df.head()

,text,url,title,content,left 0 right 1,retweet_count,favorite_count,source_name,source
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,NaN,9,63,ndtv,0
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,NaN,5,44,ndtv,0
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1.0,2,0,ndtv,0
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0.0,5,0,ndtv,0
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,NaN,3,0,ndtv,0


In [92]:
infile2 = open('republic_data.pkl','rb')
republic_df = pickle.load(infile2)

In [93]:
republic_df.count()


text              2196
url               2196
title             2196
content           2196
left 0 right 1    1297
retweet_count     2196
favorite_count    2196
source_name       2636
source            2636
dtype: int64

In [94]:
ndtv_df.count()


text              1371
url               1371
title             1371
content           1371
left 0 right 1     329
retweet_count     1371
favorite_count    1371
source_name       1371
source            1371
dtype: int64

In [95]:
republic_df=republic_df.head(1075)
ndtv_df=ndtv_df.head(1075)



In [96]:
republic_df.count()


text              1075
url               1075
title             1075
content           1075
left 0 right 1     176
retweet_count     1075
favorite_count    1075
source_name       1075
source            1075
dtype: int64

In [97]:
republic_df.tail()

,text,url,title,content,left 0 right 1,retweet_count,favorite_count,source_name,source
1070,Kerala HC quashes state govt order of reducing...,https://www.republicworld.com/india-news/law-a...,Kerala HC Quashes State Govt Order Of Reducing...,the kerala high court quashed the state govern...,NaN,4.0,21.0,republictv,1
1071,Lakhimpur Kheri violence: Yogi Adityanath hold...,https://www.republicworld.com/india-news/gener...,Lakhimpur Kheri Violence: Yogi Adityanath Hold...,chief minister yogi adityanath on monday held ...,1.0,7.0,72.0,republictv,1
1072,"IPL 2021: Know how KKR, PBKS, RR, &amp; MI can...",https://www.republicworld.com/sports-news/cric...,"IPL 2021: Know How KKR, PBKS, RR, & MI Can Qua...",the ongoing edition of the indian premier leag...,NaN,4.0,20.0,republictv,1
1073,Lakhimpur clash: Priyanka Gandhi protests deta...,https://www.republicworld.com/india-news/polit...,Lakhimpur Clash: Priyanka Gandhi Protests Deta...,"in a mark of silent protest, congress general ...",0.0,4.0,24.0,republictv,1
1074,Lakhimpur Kheri violence | UP Govt denies perm...,https://www.republicworld.com/india-news/gener...,Lakhimpur Kheri Violence | UP Govt Denies Perm...,punjab chief minister charanjit singh channi o...,NaN,7.0,47.0,republictv,1


In [98]:
##CHECKING NULL VALUES
print(republic_df.isna().sum())


text                0
url                 0
title               0
content             0
left 0 right 1    899
retweet_count       0
favorite_count      0
source_name         0
source              0
dtype: int64


In [99]:
print(ndtv_df.isna().sum())


text                0
url                 0
title               0
content             0
left 0 right 1    804
retweet_count       0
favorite_count      0
source_name         0
source              0
dtype: int64


In [100]:
republic_df=republic_df.dropna(subset=['content'])
ndtv_df=ndtv_df.dropna(subset=['content'])


In [101]:
##bias=left 0 right 1 unbiased 3

In [102]:
ndtv_df.rename(columns = {'left 0 right 1':'bias'}, inplace = True)
ndtv_df=ndtv_df.fillna({'bias':3})
ndtv_df['bias']=ndtv_df['bias'].astype(int)

In [103]:
republic_df.columns

Index(['text', 'url', 'title', 'content', 'left 0 right 1', 'retweet_count',
       'favorite_count', 'source_name', 'source'],
      dtype='object')

In [104]:
republic_df.rename(columns = {'left 0 right 1':'bias'}, inplace = True)
republic_df=republic_df.fillna({'bias':3})
republic_df['bias']=republic_df['bias'].astype(int)

In [105]:
print(republic_df.isna().sum())


text              0
url               0
title             0
content           0
bias              0
retweet_count     0
favorite_count    0
source_name       0
source            0
dtype: int64


In [106]:
print(ndtv_df.isna().sum())


text              0
url               0
title             0
content           0
bias              0
retweet_count     0
favorite_count    0
source_name       0
source            0
dtype: int64


In [107]:
republic_df.bias.value_counts()

3    899
0     93
1     83
Name: bias, dtype: int64

In [108]:
ndtv_df.bias

0       3
1       3
2       1
3       0
4       3
       ..
1070    3
1071    3
1072    3
1073    3
1074    3
Name: bias, Length: 1075, dtype: int64

# DATA CLEANING

In [109]:
df=pd.concat([ndtv_df,republic_df])

In [110]:
#REINDEXING
indx=[]
for i in range(len(df)):
  indx.append(i)
df.index=indx

# STOPWORDS

In [111]:
# A list of common english words which should not affect predictions
stopwordslist = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone',
             'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount',
             'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around',
             'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before',
             'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both',
             'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de',
             'describe', 'detail', 'did', 'do', 'does', 'doing', 'don', 'done', 'down', 'due', 'during', 'each', 'eg',
             'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone',
             'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for',
             'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had',
             'has', 'hasnt', 'have', 'having', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed',
             'interest', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less',
             'ltd', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly',
             'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine',
             'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once',
             'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
             'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed', 'seeming',
             'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 
             'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system',
             't', 'take', 'ten', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there',
             'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thickv', 'thin', 'third', 'this',
             'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward',
             'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we',
             'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby',
             'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom',
             'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself',
             'yourselves','state','said','saturday','thursday','wednesday','monday','tuesday','sunday','friday','like','power','incident','crore','added','car','arrested','visit']#,'told','government'],'leader','farmer','police'

In [112]:
final_stop_words =list(set( stopwords.words('english')+stopwordslist+['including','seat','family','time','new','son','people','country','state','mr','day','october','year','india','an','today','ndtv', 'ndtvfeed','republic','tv','republictv','story','edited','staff','syndicated','published','headline','feed']))

In [113]:
#'leader','chief','party','national','union','case','incident','violence'

In [114]:
final_stop_words.sort()

In [115]:
#final_stop_words

In [116]:
for x in final_stop_words:
  if x=='day':
    print('assajkajd')
    break

assajkajd


In [117]:
df.head(5)

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0


In [118]:
print(len(final_stop_words ))


421


In [119]:
re.sub('[.]', ' ', '.......sss')

'       sss'

In [120]:
### Dataset Preprocessing
corpus = []
for i in range(0, len(df)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', df['content'][i])
    review = re.sub('[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]"',' ',review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in final_stop_words]
    review = ' '.join(review)
    corpus.append(review)
df['clean_content']=corpus

In [121]:
df['clean_content'][0]


'army officer personnel killed action encounter security forces terrorists jammu kashmir poonch district officials anti insurgency operation launched village close dera ki gali surankote early hours following intelligence inputs presence heavily armed terrorists hiding terrorists fired search parties resulting fierce gunfight critical injuries junior commissioned officer jco soldiers died injuries encounter terrorists going details awaited defence spokesperson officials reports presence group heavily armed terrorists chamrer forest managed sneak line control loc reinforcements rushed area plug escape routes terrorists inputs pti'

In [122]:
lemmer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
df['text_lemmatized'] = df.clean_content.apply(lemmatize_text)

In [123]:
#df['text_lemmatized'][0]

In [124]:
def cleaning_lemmatize_text(text):
    #return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return[word for word in text if not word in final_stop_words]

In [125]:
df['clean_text_lemmatized'] = df.text_lemmatized.apply(cleaning_lemmatize_text)

In [126]:
#df['clean_text_lemmatized'][0]


In [127]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if x=='wa':
      print(i)
      #break

In [128]:
###TESTING
for i in range(len(df)):
  for w in (df['clean_text_lemmatized'][i]):
    #print(w)
    if(w=='car'):
      print(i)
      break

In [129]:
# df['clean_content'][1]

In [130]:
###TESTING
for i in range(len(df)):
  for w in df['clean_text_lemmatized'][i]:
    if(w=='also'):
      print(i)
      break

In [131]:
import string
from string import punctuation

In [132]:
###TESTING
re.sub('[^a-zA-Z'+string.punctuation+']', ' ','abcm,.jasjsa')
re.sub(r'[^a-zA-Z,.!(){}]', ' ','#$%&@~ !! AA @@abcm,.jasjsa')

'       !! AA   abcm,.jasjsa'

In [133]:
### Dataset Preprocessing ###
###'clean_content_with_Stopwords' we are creating this because the scraped data had special characters like 'š','â' , we need this data for sentiment analysis
stop_words = final_stop_words
from nltk.stem.porter import PorterStemmer
import string
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = re.sub(r'[^a-zA-Z,.!(){}]'+']', ' ',df['content'][i])
    #review = re.sub('[#$%&@~!_]', ' ',  ndtv_df['content'][i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    corpus.append(review)
df['clean_content_with_Stopwords']=corpus

In [134]:
df['clean_content_with_Stopwords'][0]


"an army officer and four other personnel were killed in action during an encounter between security forces and terrorists in jammu and kashmir's poonch district on monday, officials said. an anti-insurgency operation was launched in a village close to dera ki gali in surankote in the early hours following intelligence inputs about the presence of heavily-armed terrorists there, they said. the hiding terrorists fired on the search parties resulting in fierce gunfight and critical injuries to a junior commissioned officer or jco and four other soldiers. they died from their injuries. the encounter with the terrorists was going on and further details are awaited, a defence spokesperson said. officials said there were reports about the presence of a group of heavily-armed terrorists in the chamrer forest, after they managed to sneak from across the line of control (loc). reinforcements have been rushed to the area to plug all escape routes of the terrorists, they said. (with inputs from p

In [135]:
###TESTING
for i in range(len(df)):
  for w in df['clean_text_lemmatized'][i]:
    if(w=='wa'):
      print(i)
      break

In [136]:
df.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail arrest week ago drugs cruise c...,"[aryan, khan, bail, arrest, week, ago, drug, c...","[aryan, khan, bail, arrest, week, ago, drug, c...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united st...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world leaders agencies posted wishes m...,"[various, world, leader, agency, posted, wish,...","[various, world, leader, agency, posted, wish,...",various world leaders and un agencies posted w...


In [137]:
with open('/content/drive/MyDrive/project_data/clean_data.pkl', 'wb') as fid:
      pickle.dump(df, fid)

In [138]:
//

SyntaxError: ignored

In [ ]:
center_with_bias_words = df.loc[df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress', case=False)]
center_with_bias_words = center_with_bias_words.loc[center_with_bias_words['bias']==3]
right = df.loc[df['bias']==1]
left = df.loc[df['bias']==0]
center = df.loc[df['bias']==3]
df_center_with_bias_words=pd.concat([left,right,center_with_bias_words])


In [ ]:
//

#MIT News Classify

In [ ]:
# MIT News Classify is a package containing several natural language processing (NLP) and ML-based models that have been finetuned on an Annotated Corpus and its predefined news tags.

In [ ]:
df['content'][0]

In [ ]:
mitx.gettags(df['content'][0])

In [ ]:
df['content'][1]

In [ ]:
mitx.gettags(df['content'][1])


In [ ]:
mitx.gettags(df.loc[df['bias']==3]['content'][0])

In [ ]:
# ##the MIT PACKAGE IDENTIFIES TAGS AS BOTH THE ORIGINAL TAG AND WITH UNITED TAGS
# for ex.
# ['united states politics and government',
#  'politics and government',
#  'united states international relations',
#  'international relations']
#  this can be just 
#  ['politics and government',
#  'international relations']

In [ ]:
listx=[]
for x1 in range(len(df)):
  templist=[]
  test=mitx.gettags(df['content'][x1])
  for temp in test:
    if temp.find("united states") != -1:
      continue
    else:
      templist.append(temp)
  listx.append(templist)

In [ ]:
df['tags']=listx

In [ ]:
df.head()

# Sentiment Analysis

In [ ]:
# VADER works better with things like slang, emojis, etc — whereas TextBlob performs strongly with more formal language usage.Therefore we will use TextBlob.
from textblob import TextBlob

In [ ]:
(df['clean_content_with_Stopwords'][0])

In [ ]:
TextBlob(df['clean_content_with_Stopwords'][0]).sentiment.subjectivity
#TextBlob(ndf['content'][0]).sentiment.subjectivity


In [ ]:
TextBlob(df['clean_content_with_Stopwords'][0]).sentiment.polarity

Polarity

In [ ]:
listx=[]
for x1 in range(len(df)):
  listx.append(TextBlob(df['clean_content_with_Stopwords'][x1]).sentiment.polarity)
df['polarity']=listx


In [ ]:
def getAnalysis(score):
  if score < 0:
    return "Negative"
  elif score == 0:
    return "Neutral"
  else:
    return "Positive"

In [ ]:
df["polarity_score"] = df["polarity"].apply(getAnalysis)


In [ ]:
df.head(5)

Subjectivity

In [ ]:
listx=[]
for x1 in range(len(df)):
  listx.append(TextBlob(df['clean_content_with_Stopwords'][x1]).sentiment.subjectivity)
df['subjectivity']=listx


In [ ]:
df.head()

In [ ]:
def getsubAnalysis(score):
  if score > 0.5:
    return "High subjectivity"
  else:
    return "Low subjectivity"

In [ ]:
df["subjectivity_score"] = df["subjectivity"].apply(getsubAnalysis)


In [ ]:
df.head()

In [ ]:
with open('/content/drive/MyDrive/project_data/data_with_classification_and_sentiment.pkl', 'wb') as fid:
     pickle.dump(df, fid)

In [ ]:
center_with_bias_words = df.loc[df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress', case=False)]
center_with_bias_words = center_with_bias_words.loc[center_with_bias_words['bias']==3]
right = df.loc[df['bias']==1]
left = df.loc[df['bias']==0]
center = df.loc[df['bias']==3]
df_center_with_bias_words=pd.concat([left,right,center_with_bias_words])


# VISUALIZATION

In [ ]:
def common_words_plot(df, political_affiliation, first_):
    stop_words=final_stop_words
    word_list = []
    lemmer = WordNetLemmatizer()
    for row in tqdm(df):
        words = [w for w in word_tokenize(row) if (w not in final_stop_words)]
        words = [lemmer.lemmatize(w) for w in words if (len(w) > 2)]
        word_list.append(words)
    word_list = [word for l in word_list for word in l]
    counter = Counter(word_list)
    common_ = counter.most_common()
    word = []
    count = []
    for common_word, word_count in common_:
        if common_word not in stop_words:
            word.append(common_word)
            count.append(word_count)
         #print(common_word)
    plt.figure(figsize=(15,9))
    sns.barplot(word[:first_], count[:first_]).set_title(f'Most Common Words from the {political_affiliation}',fontsize=20)
    plt.xlabel("Words",fontsize=15)
    plt.xticks(fontsize=15)
    plt.ylabel("Counts",fontsize=15)
    plt.yticks(fontsize=15)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
from nltk.util import ngrams
def n_gram_plot(df, n, political_affiliation, first_):
    stop_words=final_stop_words
    print('vectorizing...')
    vec = CountVectorizer(ngram_range=(n, n),stop_words=final_stop_words).fit(df)
    print('transforming...')
    bag_of_words = vec.transform(df)
    sum_words = bag_of_words.sum(axis=0)
    print('building frequency...')
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    x,y=map(list,zip(*words_freq))
    grams = 'Bigrams'
    if n == 3: grams = 'Trigrams'
    if n == 4: grams = 'Quadgrams'
    print('plotting...')
    plt.figure(figsize=(15,9))
    sns.barplot(y[:first_],x[:first_],palette="GnBu_d").set_title(f'Most Common {grams} from the {political_affiliation}',fontsize=20)
    plt.xlabel("Bigrams",fontsize=15)
    plt.xticks(fontsize=15)
    plt.ylabel("Counts",fontsize=15)
    plt.yticks(fontsize=15)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
common_words_plot(left['clean_content_with_Stopwords'],'left',30)



In [ ]:
common_words_plot(right['clean_content_with_Stopwords'],'right',30)


In [ ]:
common_words_plot(center_with_bias_words['clean_content_with_Stopwords'],'center with biased words',30)

In [ ]:
common_words_plot(center['clean_content_with_Stopwords'],'center with biased words',30)

#BIGRAM COMMON WORDS

In [ ]:
##BIGRAM COMMON WORDS
n_gram_plot(left['clean_content_with_Stopwords'],2,'left',20)

In [ ]:
n_gram_plot(right['clean_content_with_Stopwords'],2,'right',20)

In [ ]:
n_gram_plot(center_with_bias_words['clean_content_with_Stopwords'],2,'center with biased words',20)

In [ ]:
n_gram_plot(center['clean_content_with_Stopwords'],2,'center without biased words',20)

# TRIGRAM COMMON WORDS

In [ ]:
##TRIGRAM COMMON WORDS
n_gram_plot(left['clean_content_with_Stopwords'],3,'left',30)

In [ ]:
n_gram_plot(right['clean_content_with_Stopwords'],3,'right',30)

In [ ]:
n_gram_plot(center_with_bias_words['clean_content_with_Stopwords'],3,'center with biased words',30)

In [ ]:
n_gram_plot(center['clean_content_with_Stopwords'],3,'center with biased words',30)

# WORDCLOUD

In [ ]:
from wordcloud import WordCloud
###LEFT WORDCLOUD
text = left['clean_content'].values 
wordcloud = WordCloud(width=800, height=400).generate(str(text))
plt.figure(figsize = (12, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
###RIGHT WORDCLOUD
text = right['clean_content'].values 
wordcloud = WordCloud(width=800, height=400).generate(str(text))
plt.figure(figsize = (12, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
###CENTER WITH BIAS WORDS WORDCLOUD
text = center_with_bias_words['clean_content'].values 
wordcloud = WordCloud(width=800, height=400).generate(str(text))
plt.figure(figsize = (12, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
###CENTER WORDCLOUD
text = center['clean_content'].values 
wordcloud = WordCloud(width=800, height=400).generate(str(text))
plt.figure(figsize = (12, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
##ALL WITH THE TWITTER DATA
##RETWEETS
fig = plt.figure(figsize = (10, 5))
 
tweets = df['retweet_count']
bias= df['bias']
# creating the bar plot
plt.figure(figsize = (8, 7), facecolor = None)
plt.xticks([0,1,3],['left','right','center'])
plt.scatter(bias,tweets)
plt.show()

In [ ]:
##ALL WITH THE TWITTER DATA
##favourite count
fig = plt.figure(figsize = (10, 5))
tweets = df['favorite_count']
bias= df['bias']
# creating the bar plot
plt.figure(figsize = (8, 7), facecolor = None)
plt.xticks([0,1,3],['left','right','center'])
plt.scatter(bias,tweets)
plt.show()

In [ ]:
with open('/content/drive/MyDrive/project_data/model_data.pkl', 'wb') as fid:
      pickle.dump(df, fid)

In [ ]:
df.loc[df['clean_content'].str.contains('india')]#, case=False)]

In [ ]:
df['clean_content'][2]

In [ ]:
###TESTING
from nltk.tokenize import word_tokenize
# create bag-of-words
all_words = []
for i in range(len(df)):
  for w in df['clean_text_lemmatized'][i]:
    if(w=='also'):
      print(i)
      break
        #all_words.append(w)
        
#all_words = nltk.FreqDist(all_words)

In [ ]:
///

# MODEL


In [ ]:
cd /content/drive/MyDrive/project_data

In [ ]:
infile = open('model_data.pkl','rb')
df = pickle.load(infile)

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# max_features = 30000
# maxlen = 25
# X = df['clean_text_lemmatized']
# X = pad_sequences(X, maxlen = maxlen)
# y=df['bias']

In [ ]:
//

In [ ]:
center_with_bias_words = df.loc[df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress', case=False)]
center_without_bias_words = df.loc[~df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress', case=False)]

center_without_bias_words = center_without_bias_words.loc[center_without_bias_words['bias']==3]

center_with_bias_words = center_with_bias_words.loc[center_with_bias_words['bias']==3]
right = df.loc[df['bias']==1]
left = df.loc[df['bias']==0]
#center = df.loc[df['bias']==3]

#df2=pd.concat([left,right,center_with_bias_words])


In [ ]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



In [ ]:
left=left.head(190)

In [ ]:
#center_with_bias_words=center_with_bias_words.head(100)

In [ ]:
#center_without_bias_words=center_without_bias_words.head(100)


In [ ]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



In [ ]:
df2=pd.concat([left,right,center_with_bias_words,center_without_bias_words])


In [ ]:
#REINDEXING
indx=[]
for i in range(len(df2)):
  indx.append(i)
df2.index=indx

In [ ]:
df2

In [ ]:
# ## Divide the dataset into Train and Test
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [ ]:
######NaiiveBayes without Feature Selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score,classification_report,accuracy_score,confusion_matrix
from sklearn.naive_bayes import GaussianNB
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df2['clean_content']
y=df2['bias']


In [ ]:
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=20,ngram_range=(1,3))

In [ ]:
X=tfidf_v.fit_transform(df2['clean_content']).toarray()


In [ ]:
X

In [ ]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [ ]:
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())
count_df.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np
import itertools
classifier=MultinomialNB()
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
cm
#plot_confusion_matrix(cm, classes=pred.c)##bias=left 0 right 1 unbiased 3
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


# RF  Manual Hyperparameter Tuning

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
model=RandomForestClassifier(n_estimators=900,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=1,min_samples_split=2,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

#RF WITH RANDOMSEARCH CV

RF WITH RANDOMSEARCH CV

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 500,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

In [ ]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=10,cv=3,verbose=2,
                               random_state=50,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

In [ ]:
rf_randomcv.best_params_

In [ ]:
best_random_grid=rf_randomcv.best_estimator_
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

#RF WITH GridSearchCV


RF WITH GridSearchCV


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)


# Multinomial with Hyperparameter


In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

# RF WITH TPOT OPTIMIZATION

In [ ]:
pip install tpot


In [ ]:
from tpot import TPOTClassifier

tpot_classifier = TPOTClassifier(generations= 3, population_size= 30, offspring_size= 10,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier':param}, 
                                 cv = 4, scoring = ['accuracy','recall','precision'])
tpot_classifier.fit(X_train,y_train)

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
#model=RandomForestClassifier( criterion='gini', max_depth=340, max_features='auto', min_samples_leaf=2, min_samples_split=5, n_estimators=2000).fit(X_train,y_train)
model=RandomForestClassifier(criterion='entropy', max_depth=780, max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
pip install optuna


In [ ]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

In [ ]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=text_clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))